<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Using different flow directions

This notebook starts from the basic example of the [first notebook](1_basic-example.ipynb) to show how to use Landlab's flow directors with StratigraPy.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmocean

from landlab.components import FlowDirectorSteepest, FlowDirectorDINF, FlowAccumulator

from stratigrapy import RasterModelGrid
from stratigrapy.components import CyclicSeaLevelCalculator, WaterDrivenRouter

## 1. Steepest-path flow direction

We'll use the same case as in the [first notebook](1_basic-example.ipynb), starting with the same simulation time:

In [ ]:
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

Then the grid:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100,
                       fuse_continuously=True)

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

The initial topography:

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

The sources of water and sediments:

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

The component controlling sea level:

In [ ]:
slc = CyclicSeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

The only change compared to the orginal cases: we use a steepest-path flow director instead of a multiple-path flow director.

In [ ]:
fd = FlowDirectorSteepest(grid)

In [ ]:
fa = FlowAccumulator(grid, flow_director=fd)

Finally, the component controlling sediment transport:

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

We can now run the model:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

And visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='x (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', cmap=cmocean.cm.deep, zorder=2)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

## 2. D-infinity flow direction

We can just follow the same principle with the other flow directors available in Landlab, for instance with the `FlowDirectorDINF`, this time grouping all the necessary steps into a single cell:

In [ ]:
# Define the number of iterations
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

# Define the grid
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100,
                       fuse_continuously=True)
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

# Define the initial topography
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

# Define the sources of water and sediments
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

# Define the components for sea level variation, water flow, and sediment transport
slc = CyclicSeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])
fd = FlowDirectorDINF(grid)
fa = FlowAccumulator(grid, flow_director=fd)
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

# Run the simulation
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

And again we can visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='x (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', cmap=cmocean.cm.deep, zorder=2)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));